In [1]:
using Flux.Tracker
using Flux
using Flux:throttle,glorot_uniform
using BSON:@save
using BSON:@load
using Base.Iterators: repeated
using Flux:@treelike
using PyPlot
using FastGaussQuadrature
using SparseGrids
using LinearAlgebra
using Flux: @epochs
using DataFrames
using CSV
using Sobol
using Pkg; Pkg.activate("cuda"); Pkg.instantiate()
using CuArrays
using ForwardDiff
using CUDAnative
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float32,1}, ::Val{2}) = x*x
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float64,1}, ::Val{2}) = x*x

f(x)=2*π^2 .*sum(cos.(π*x[i,:]') for i=1:2, dims=1)
acti(x)=@. x/(1+CUDAnative.exp(-x))


struct Block{F,S,T}
  W1::S
  W2::S
  b1::T
  b2::T
  σ::F
end

Block(W1 , W2, b1, b2) = Block(W1,W2 ,b1 ,b2, identity)

function Block(in1::Integer,in2::Integer, out1::Integer,out2::Integer, σ = identity;
  initW1 = glorot_uniform, initW2 =glorot_uniform, initb1 = zeros,initb2=zeros)
  return Block(param(initW1(out1, in1)),param(initW2(out2,in2)), param(initb1(out1)), param(initb2(out2)),σ)
end

@treelike Block

function (a::Block)(x)
  W1,W2, b1,b2, σ = a.W1,a.W2,a.b1, a.b2, a.σ
  σ.(W2 *σ.(W1*x .+ b1) .+ b2) .+ x
end




M=Chain(
Dense(2,10),
Block(10,10,10,10,acti),
Block(10,10,10,10,acti),
Block(10,10,10,10,acti),
Block(10,10,10,10,acti),
Dense(10,1)
)|>gpu

function lossFD(N)
  notes=zeros(2,N)
  for i=1:N
    notes[:,i]=next!(inner)
  end
  notes =notes |>gpu
  d=rand(1)/100 |>gpu
  losses=0
  MMM=M(notes)
  for i=1:2
    l1=zeros(2)
    l1[i]=1
    l1 = l1 |>gpu
    losses  += (sum(0.5*((M(notes .+ d.*l1) .- MMM)./d).^2)/N)[1]
  end
  losses += (sum(0.5*(π^2 * (MMM).^2))/N)[1]
  losses += (sum(-f(notes) .*MMM)/N)[1]
  return losses
end



function lossBC(β)
  d=(rand(1)/100)|>gpu
  point=zeros(2,400)
  p=zeros(1,100)
  for i=1:100
    p[:,i]=next!(BC)
  end
  for i=1:2
    point[:,200*(i-1)+1:200*(i-1)+100]=vcat(p[1:i-1,:],zeros(100)',p[i:end,:])
    point[:,200*(i-1)+101:200*i]=vcat(p[1:i-1,:], ones(100)',p[i:end,:])
  end
    point=point|>gpu
  losses=0
  for i=1:2
    l2=zeros(2)
    l1=zeros(2)
    l2[i]=2
    l1[i]=1
    l1=l1|>gpu
    l2=l2|>gpu
    losses +=β*(sum((( 1/2*M(point[:,200*(i-1)+1:200*i] .+ d.*l1) .- 1/2*M(point[:,200*(i-1)+1:200*i] .- d.*l1))./d).^2)/100)[1]
  end
  return losses
end

function loss1(N)
  return lossBC(50)+lossFD(N)
end

function loss_true(xx=1,yy=1,xx1=1,xx2=2)
  points=rand(2,2000)|>gpu
  F_true(x)= sum(cos.(pi*x[i,:]') for i=1:2, dims=1)
  errors=sqrt(sum((M(points)-F_true(points)).^2)/2000)
  return errors
end

inner=SobolSeq(2)
BC=SobolSeq(1)
function test(N;traintime=2000)
  errdf=DataFrame(time=Int[],absolute_error=Float64[],loss_FD=Float64[],loss_BC=Float64[])
  cntr=1
  evalcb = function()
    loss_true1=loss_true()
    loss_FD = lossFD(N)
    loss_BC =lossBC(1)
    push!(errdf,[Tracker.data(cntr),Tracker.data(loss_true1),Tracker.data(loss_FD),Tracker.data(loss_BC)])
    cntr+=1
    if cntr%1000==0
      @show(cntr,loss_FD,loss_true1,loss_BC)
      CSV.write("$(N)Sobol2Dexm2.csv",errdf)
      @save "$(N)Sobolpoint2Dexm2m.bson" M
      weights=Tracker.data.(params(M))
      @save "$(N)Sobolpoint2Dexm2w.bson" weights
    end
  end
  θ=Flux.params(M)
  opt=ADAM()
  dataset=[(N) for i=1:traintime]
  Flux.train!(loss1, params(M), zip(dataset), opt, cb=evalcb)
  return errdf
end


Activating new environment at `~/2Dexm2/cuda/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


test (generic function with 1 method)

In [ ]:
errdf=test(2000,traintime=100000)

cntr = 1000
loss_FD = -9.76602981935348 (tracked)
loss_true1 = 0.06969719f0 (tracked)
loss_BC = 0.0014347637375686107 (tracked)
cntr = 2000
loss_FD = -9.872906838695403 (tracked)
loss_true1 = 0.028960463f0 (tracked)
loss_BC = 0.0013597196784803807 (tracked)
cntr = 3000
loss_FD = -9.717993962083948 (tracked)
loss_true1 = 0.013364814f0 (tracked)
loss_BC = 0.0003915841370996344 (tracked)
cntr = 4000
loss_FD = -9.624079233030855 (tracked)
loss_true1 = 0.016146237f0 (tracked)
loss_BC = 0.00010733699271900886 (tracked)
cntr = 5000
loss_FD = -9.704745095942641 (tracked)
loss_true1 = 0.024307892f0 (tracked)
loss_BC = 0.00015557087816910336 (tracked)
cntr = 6000
loss_FD = -9.697172162256411 (tracked)
loss_true1 = 0.011735486f0 (tracked)
loss_BC = 0.00012365115957637123 (tracked)
cntr = 7000
loss_FD = -9.767546155719 (tracked)
loss_true1 = 0.012821011f0 (tracked)
loss_BC = 5.582493335626759e-5 (tracked)
cntr = 8000
loss_FD = -9.891887752357984 (tracked)
loss_true1 = 0.011881498f0 (tracked)
loss_

cntr = 64000
loss_FD = -9.866300441472362 (tracked)
loss_true1 = 0.019734105f0 (tracked)
loss_BC = 2.33473316355752e-5 (tracked)
cntr = 65000
loss_FD = -9.8232807779365 (tracked)
loss_true1 = 0.0071509825f0 (tracked)
loss_BC = 0.00022582554678609313 (tracked)
cntr = 66000
loss_FD = -9.70822078561702 (tracked)
loss_true1 = 0.010262896f0 (tracked)
loss_BC = 1.152518774380815e-5 (tracked)
cntr = 67000
loss_FD = -9.802467325950465 (tracked)
loss_true1 = 0.007901633f0 (tracked)
loss_BC = 8.53788250028243e-6 (tracked)
cntr = 68000
loss_FD = -10.001593692226315 (tracked)
loss_true1 = 0.009093981f0 (tracked)
loss_BC = 0.0004326783227347489 (tracked)
cntr = 69000
loss_FD = -9.710089786446408 (tracked)
loss_true1 = 0.00467294f0 (tracked)
loss_BC = 1.9939649367345374e-5 (tracked)
cntr = 70000
loss_FD = -9.881789790813295 (tracked)
loss_true1 = 0.0028245063f0 (tracked)
loss_BC = 0.0001943885984611525 (tracked)
cntr = 71000
loss_FD = -9.781015072275506 (tracked)
loss_true1 = 0.007979096f0 (tracked)

cntr = 27000
loss_FD = -9.855084610735524 (tracked)
loss_true1 = 0.005386224f0 (tracked)
loss_BC = 0.00014223302496657987 (tracked)
cntr = 28000
loss_FD = -9.931028025668724 (tracked)
loss_true1 = 0.0042834976f0 (tracked)
loss_BC = 2.9299665726012035e-5 (tracked)
cntr = 29000
loss_FD = -9.914535111463628 (tracked)
loss_true1 = 0.0060418183f0 (tracked)
loss_BC = 0.0008968432560774509 (tracked)
cntr = 30000
loss_FD = -9.92316084941503 (tracked)
loss_true1 = 0.0048824246f0 (tracked)
loss_BC = 7.903011465311223e-6 (tracked)
cntr = 31000
loss_FD = -9.92720555066061 (tracked)
loss_true1 = 0.0050465534f0 (tracked)
loss_BC = 7.704350044694842e-5 (tracked)
cntr = 32000
loss_FD = -9.92528507864509 (tracked)
loss_true1 = 0.0052070604f0 (tracked)
loss_BC = 4.727520355191894e-6 (tracked)
cntr = 33000
loss_FD = -9.920651089474333 (tracked)
loss_true1 = 0.00425707f0 (tracked)
loss_BC = 5.932842040970478e-5 (tracked)
cntr = 34000
loss_FD = -9.966063356171556 (tracked)
loss_true1 = 0.0054589813f0 (trac